# Sequences by Functional Programming
Computers in the real world have finitely many states, and so they are restricted to operating on finite sets.
Yet these limitations allow for some interpretation.
We use basic topics in calculus as guiding examples. 

## Sequences
By definition a <i>sequence</i> of reals is a map from $\mathbb{N}$ to $\mathbb{R}$.
This can be written as a Julia function quite directly.

In [1]:
a = (n -> 1/n)

#1 (generic function with 1 method)

This is how to look at some specific coefficients.  In the Julia universe indices start with one; hence we let $\mathbb{N}=\{1,2,3\dots\}$.  This deviates from our usual course conventions.  The reals are approximated by (64-bit) floating point numbers.

In [2]:
map(a, [1,2,3])

3-element Vector{Float64}:
 1.0
 0.5
 0.3333333333333333

Now we can write functions which operate on sequences.  Here is the sum of $a$ and $b$.

In [3]:
function add(a,b)
    return (n -> a(n)+b(n))
end

add (generic function with 1 method)

Notice that writing functions via function/return/end or in the "->" notation does not make a difference in Julia; the user can choose.  Here, we reserve the "->" notation for the sequences themselves; these are the (function) objects that we wish to study.

We define a second sequence, and combine all of the above.

In [4]:
b = (n->n^2)
map(add(a,b), 1:3)

3-element Vector{Float64}:
 2.0
 4.5
 9.333333333333334

## Series
A <i>series</i>, with given coefficients, is the sequence of the partial sums.

In [5]:
series(a) = (n -> sum(a,1:n))

series (generic function with 1 method)

Series are nothing but ordinary sequences, written in a special way.

In [6]:
map(series(a), 1:3)

3-element Vector{Float64}:
 1.0
 1.5
 1.8333333333333333

Here are two standard examples which are discussed in every calculus class.

In [7]:
harmonic_series = series(n->1/n)
geometric_series = (q -> series(n->q^n))

#11 (generic function with 1 method)

In [8]:
map(harmonic_series, 1:5)

5-element Vector{Float64}:
 1.0
 1.5
 1.8333333333333333
 2.083333333333333
 2.283333333333333

In [9]:
map(geometric_series(1/2), 1:5)

5-element Vector{Float64}:
 0.5
 0.75
 0.875
 0.9375
 0.96875

The harmonic series diverges, while the geometric series converges for any real $q$ with $|q|<1$.  We can see a glimpse of that convergence, but this is not a formal proof.  To the contrary, technically this is an artifact owing to the limits of machine precision.

In [10]:
geometric_series(1/2)(100)==1

true

## Riemann Series Theorem

We consider a sequence $a:\mathbb{N}\to\mathbb{R}$ such that the induced series converges but not absolutely.  Further, let $x\in\mathbb{R}$.

<b>Theorem.</b>
With these assumptions there is a bijection $\phi:\mathbb{N}\to\mathbb{N}$ such that
$
\sum_{n=1}^\infty a_{\phi(n)} = x
$.

Before we will look into a constructive proof of that result, we introduce a standard example.

In [11]:
alternating_harmonic_coeffs = n -> (-1)^(n-1)/n

map(alternating_harmonic_coefNowfs, 1:10)

10-element Vector{Float64}:
  1.0
 -0.5
  0.3333333333333333
 -0.25
  0.2
 -0.16666666666666666
  0.14285714285714285
 -0.125
  0.1111111111111111
 -0.1

The induced series converges $\ldots$

In [12]:
map(series(alternating_harmonic_coeffs), 101:110)

10-element Vector{Float64}:
 0.6980731694092049
 0.6882692478405775
 0.6979779857046552
 0.6883626010892706
 0.6978864106130801
 0.6884524483489292
 0.6977982427414525
 0.6885389834821932
 0.6977132954087988
 0.6886223863178897

$\ldots$ and this is the limit:

In [13]:
log(2)

0.6931471805599453

We define two auxiliary function which will be used below.  For a given sequence $a$ and an index $i$ they find the first index $k\geq i$ such that $a_k>0$ (or $a_k<0$, respectively).

In [14]:
function next_positive_index(a,i)
    k = i
    while a(k)<=0Now
        k += 1
    end
    return k
end

function next_negative_index(a,i)
    k = i
    while a(k)>=0
        k += 1
    end
    return k
end

next_negative_index (generic function with 1 method)

The functions above and below do <i>not</i> follow traditional rules of pure functional programming; it is an advantage of Julia (and other languages) that functional concepts can be mixed with, e.g., imperative programming.  Here imperative programming yields code which is very close to standard text book proofs in calculus.

Now we assume that $a$ and $x$ meet the assumption of the theorem above.  That is, $a$ is a sequence such that the series of partial sums converges but not absolutely, and $x$ is a real number to converge to for the reordering.
The function below returns the first $k$ indices of the reordering bijection $\phi$.

In [15]:
function riemann(a,x,k)
    partialsum = 0
    i = 1
    ipos,ineg = 1,1 # candidates for next pos/neg index
    phi = []
    for j = 1:k
        if partialsum < x
            i = next_positive_index(a,ipos)
            ipos = i+1
        else
            i = next_negative_index(a,ineg)
            ineg = i+1
        end
        push!(phi,i)
        partialsum += a(i)
    end
    return phi
end


riemann (generic function with 1 method)

This is how the first 100 values of the permutation $\phi$ look like for the alternating harmonic series.

In [16]:
phi = riemann(alternating_harmonic_coeffs,0.1,100);
@show phi;

phi = Any[1, 2, 4, 6, 3, 8, 10, 12, 14, 5, 16, 18, 20, 7, 22, 24, 26, 9, 28, 30, 32, 11, 34, 36, 38, 40, 13, 42, 44, 46, 15, 48, 50, 52, 17, 54, 56, 58, 19, 60, 62, 64, 66, 21, 68, 70, 72, 23, 74, 76, 78, 25, 80, 82, 84, 86, 27, 88, 90, 92, 29, 94, 96, 98, 31, 100, 102, 104, 33, 106, 108, 110, 112, 35, 114, 116, 118, 37, 120, 122, 124, 39, 126, 128, 130, 132, 41, 134, 136, 138, 43, 140, 142, 144, 45, 146, 148, 150, 47, 152]


These are the actual coefficents and their sum.

In [17]:
Now@show map(alternating_harmonic_coeffs, phi);
@show sum(alternating_harmonic_coeffs, phi);

map(alternating_harmonic_coeffs, phi) = [1.0, -0.5, -0.25, -0.16666666666666666, 0.3333333333333333, -0.125, -0.1, -0.08333333333333333, -0.07142857142857142, 0.2, -0.0625, -0.05555555555555555, -0.05, 0.14285714285714285, -0.045454545454545456, -0.041666666666666664, -0.038461538461538464, 0.1111111111111111, -0.03571428571428571, -0.03333333333333333, -0.03125, 0.09090909090909091, -0.029411764705882353, -0.027777777777777776, -0.02631578947368421, -0.025, 0.07692307692307693, -0.023809523809523808, -0.022727272727272728, -0.021739130434782608, 0.06666666666666667, -0.020833333333333332, -0.02, -0.019230769230769232, 0.058823529411764705, -0.018518518518518517, -0.017857142857142856, -0.017241379310344827, 0.05263157894736842, -0.016666666666666666, -0.016129032258064516, -0.015625, -0.015151515151515152, 0.047619047619047616, -0.014705882352941176, -0.014285714285714285, -0.013888888888888888, 0.043478260869565216, -0.013513513513513514, -0.013157894736842105, -0.01282051282051282, 

That's close to $x=0.1$, but not very close.  This is a bit better:

In [18]:
phi = riemann(alternating_harmonic_coeffs,0.1,1000);
sum(alternating_harmonic_coeffs, phi)

0.09989073409781402

Yet the convergence is quite slow.

In [19]:
phi = riemann(alternating_harmonic_coeffs,0.1,10000);
sum(alternating_harmonic_coeffs, phi)

0.10018402080765942

There are some Julia packages which deal with sequences in some way.  I might add references later.